# Deep Products: Deep Tag Labeler

This is the first project for the book Deep Products, about using NLP and weakly supervised learning to build complete machine learning products. Using the non-code text of Stack Overflow posts (question and answers) to tag them using a multi-class, multi-label classifier using LSTMs and Emlo embeddings.

In [1]:
import json
import os
import re

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm_notebook
#import tensorflow_addons as tfa

print(
    tf.test.is_gpu_available(
        cuda_only=False,
        min_cuda_compute_capability=None
    )
)
print()
print(
    tf.compat.v2.config.experimental.list_physical_devices('GPU')
)

True

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [2]:
np.random.seed(seed=1337)

## Load a Stratified Sample of Answered Stack Overflow Questions with Tags

We load a sample pulled from all answered questions from Stack Overflow. This data was converted from XML to parquet format via [code/stackoverflow/xml_to_parquet.py](stackoverflow/sample_json.spark.py) and then a more balanced stratified sample was computed for tags with over 50,000, 20,000 and 10,000 instances that reduced the maximum imbalance from 100-1000:1 to 8:1 using [code/stackoverflow/get_questions.spark.py](stackoverflow/get_questions.spark.py).

These scripts were run using a Spark cluster via Amazon Elastic MapReduce using 13 r5.12xlarge machines for about 24 hours at a cost of about \\$300 per full run, and about \\$1,500 overall to create and debug. Big data is expensive.

With this dataset the challenge isn't the number of records per say but rather the imbalance of the dataset if we wish to expand the number of tags the model can predict beyond low 3 digits. This leads us to some of the other techniques we'll cover involving weakly supervised learning.

In [3]:
sorted_all_tags = json.load(open('data/stackoverflow/08-05-2019/sorted_all_tags.50000.json'))
max_index = sorted_all_tags[-1][0] + 1

In [4]:
import pyarrow
posts_df = pd.read_parquet(
    'data/stackoverflow/08-05-2019/Questions.Stratified.Final.50000.parquet',
    columns=['_Body'] + ['label_{}'.format(i) for i in range(0, max_index)],
    engine='pyarrow'
)
posts_df.head(5)

,_Body,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,...,label_14,label_15,label_16,label_17,label_18,label_19,label_20,label_21,label_22,label_23
0,"[C, Mono, Winforms, MessageBox, problem, I, fi...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Are, NET, data, providers, Oracle, require, O...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[How, I, focus, foreign, window, I, applicatio...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[Default, button, hit, windows, forms, trying,...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[Can, I, avoid, JIT, net, Say, code, always, g...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
print(
    '{:,}'.format(
        len(posts_df.index)
    )
)

1,293,018


## Map from Tags to IDs

In [6]:
tag_index = json.load(open('data/stackoverflow/08-05-2019/tag_index.50000.json'))
index_tag = json.load(open('data/stackoverflow/08-05-2019/index_tag.50000.json'))

## Count the Most Common Tags

In [7]:
label_counts = json.load(open('data/stackoverflow/08-05-2019/label_counts.50000.json'))

# Sanity check the difference files
assert(len(label_counts.keys()) == len(tag_index.keys()) == len(index_tag.keys()) == len(sorted_all_tags))

## Make Record Count a Multiple of the Batch Size and Post Sequence Length

The Elmo embedding requires that the number of records be a multiple of the batch size times the number of tokens in the padded posts.

In [8]:
import math

BATCH_SIZE = 128
MAX_LEN = 100
TOKEN_COUNT = 10000
EMBED_SIZE = 50

# Convert label columns to numpy array
labels = posts_df[list(posts_df.columns)[1:]].to_numpy()

# training_count must be a multiple of the BATCH_SIZE times the MAX_LEN for the Elmo embedding layer
highest_factor = math.floor(len(posts_df.index) / (BATCH_SIZE * MAX_LEN))
training_count = highest_factor * BATCH_SIZE * MAX_LEN
print('Highest Factor: {:,} Training Count: {:,}'.format(highest_factor, training_count))

# Remove stopwords - now done in Spark, so can remove once that runs
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

documents = []
for body in posts_df[0:training_count]['_Body'].values.tolist():
    words = body.tolist()
    documents.append(' '.join(words))

labels = labels[0:training_count]

# Lengths for x and y match
assert( len(documents) == training_count == labels.shape[0])

Highest Factor: 101 Training Count: 1,292,800


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# sentences = [' '.join(x) for x in posts_text]
# sentences

## Create an Elmo Embedding Layer using Tensorflow Hub

Note that this layer takes a padded two-dimensional array of strings.

In [10]:
# # From https://www.depends-on-the-definition.com/named-entity-recognition-with-residual-lstm-and-elmo/
# tf.compat.v1.disable_eager_execution()

# sess = tf.compat.v1.Session()

# elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

# sess.run(tf.global_variables_initializer())
# sess.run(tf.tables_initializer())

# def ElmoEmbedding(x):
#     return elmo_model(inputs={
#                             "tokens": tf.squeeze(tf.cast(x, tf.string)),
#                             "sequence_len": tf.constant(BATCH_SIZE*[MAX_LEN])
#                       },
#                       signature="tokens",
#                       as_dict=True)["elmo"]

# text_input = Input(shape=(max_len,), dtype=tf.string)
# elmo_embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(text_input)

## Create a Glove Embedding Layer

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=TOKEN_COUNT)
tokenizer.fit_on_texts(documents)
# encoded_docs = tokenizer.texts_to_matrix(posts_text, mode='tfidf')
sequences = tokenizer.texts_to_sequences(documents)

padded_sequences = pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    dtype='int32',
    padding='post',
    truncating='pre',
    value=1
)

print(max([len(x) for x in padded_sequences]), min([len(x) for x in padded_sequences]))
assert( min([len(x) for x in padded_sequences]) == MAX_LEN == max([len(x) for x in padded_sequences]))

padded_sequences.shape

100 100


(1292800, 100)

In [12]:
padded_sequences

array([[  71, 3402, 2170, ...,    1,    1,    1],
       [  54,  170,   34, ...,    1,    1,    1],
       [  21,    2, 1478, ...,    1,    1,    1],
       ...,
       [   2,   13,   35, ...,  956,    1,    1],
       [ 119,  462,   32, ...,    1,    1,    1],
       [   2,   25,   44, ...,    1,    1,    1]], dtype=int32)

In [13]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open('data/GloVe/glove.6B.50d.txt'))

In [14]:
# Create embeddings matrix
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

# Create embedding matrix using our vocabulary
word_index = tokenizer.word_index
print('word_index', word_index)
nb_words = min(TOKEN_COUNT, len(word_index))

# Initialize embedding matrix
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))

# Loop through each word and get its embedding vector
missing_count = 0
too_short_count = 0
for word, i in word_index.items():
    if i >= TOKEN_COUNT: 
        too_short_count += 1
        continue # Skip words appearing less than the minimum allowed
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
    else:
        missing_count += 1

# print(missing_count, too_short_count, embedding_matrix[0])

/home/ubuntu/anaconda3/envs/deep/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


word_index 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Experimental Setup

We `train_test_split` rather than k-fold cross validate because it is too expensive.

In [15]:
from sklearn.model_selection import train_test_split

TEST_SPLIT = 0.1

# X_train, X_test, y_train, y_test = train_test_split(
#     posts_text,
#     labels,
#     test_size=TEST_SPLIT,
#     random_state=1337
# )
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences,
    labels,
    test_size=TEST_SPLIT,
    random_state=1337
)

assert(X_train.shape[0] == y_train.shape[0])
assert(X_train.shape[1] == MAX_LEN)
assert(X_test.shape[0] == y_test.shape[0]) 
assert(X_test.shape[1] == MAX_LEN)

## Start with a sub-sample

In [16]:
X_train = X_train[:100000]
y_train = y_train[:100000]
X_test  = X_test[:10000]
y_test  = y_test[:10000]

## Create an LSTM Model to Classify Posts into Tags

We use the padded/tokenized posts as input, an Elmo embedding feeding an Long-Short-Term-Memory (LSTM) layer followed by a Dense layer with the same number of output neurons as our tag list.

We use focal loss as a loss function, which is used in appliations like object detection, because it 

In [17]:
# from keras.layers import Input, concatenate, Activation, Dense, LSTM, BatchNormalization, Embedding, Dropout, Lambda, Bidirectional
# from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
# from keras.models import Model
# from keras.optimizers import Adam
# from keras_metrics import precision, f1_score, false_negative, true_positive, false_positive, true_negative

# # Text model
# text_input = Input(shape=(MAX_LEN,), dtype=tf.string)

# elmo_embedding = Lambda(ElmoEmbedding, output_shape=(MAX_LEN, 1024))(text_input)

# text_lstm = LSTM(
#     input_shape=(MAX_LEN, 1024,),
#     units=512,
#     recurrent_dropout=0.2,
#     dropout=0.2)(elmo_embedding)

# text_dense = Dense(200, activation='relu')(text_lstm)

# text_output = Dense(record_count, activation='sigmoid')(text_dense)

# text_model = Model(
#     inputs=text_input, 
#     outputs=text_output
# )



# from sklearn.metrics import hamming_loss

# from keras.optimizers import Adam
# adam = Adam(lr=0.0005)

# text_model.compile(
#     loss='binary_crossentropy',
#     optimizer=adam,
#     metrics=[
#         precision_m,
#         recall_m,
#         f1_m,
#         'mae',
#         abs_KL_div,
#         'accuracy'
#     ]
# )
# 
# text_model.summary()

## Compute Sample and Class Weights

Because we have skewed classes and multiple classes per example, we employ sample or class weights which weight the importance of each row according to the relative frequency of their labels.

In [18]:
from sklearn.utils.class_weight import compute_sample_weight

train_sample_weights = compute_sample_weight('balanced', y_train)
test_sample_weights = compute_sample_weight('balanced', y_test)

train_sample_weights, test_sample_weights

(array([1.31686345e-03, 4.83970147e-06, 3.44373899e-06, ...,
        1.31385837e-07, 1.31385837e-07, 3.73631618e-06]),
 array([1.29362929e-07, 1.21589841e-04, 1.29362929e-07, ...,
        4.27073671e-06, 7.71081461e-06, 1.29362929e-07]))

In [19]:
train_weight_vec = list(np.max(np.sum(y_train, axis=0))/np.sum(y_train, axis=0))
train_class_weights = {i: train_weight_vec[i] for i in range(y_train.shape[1])}

test_weight_vec = list(np.max(np.sum(y_test, axis=0))/np.sum(y_test, axis=0))
test_class_weights = {i: test_weight_vec[i] for i in range(y_test.shape[1])}

sorted(list(train_class_weights.items()), key=lambda x: x[1]), sorted(list(test_class_weights.items()), key=lambda x: x[1])

([(16, 1.0),
  (5, 1.204454497998956),
  (12, 1.3966908797417272),
  (17, 1.609767441860465),
  (20, 1.8677819751753912),
  (10, 1.8835374149659865),
  (9, 2.0376803061524873),
  (8, 2.0780546382467726),
  (7, 2.181531673495115),
  (13, 2.2037567653613497),
  (22, 2.210089399744572),
  (14, 2.2732348111658456),
  (1, 2.41016713091922),
  (11, 2.4304775280898876),
  (23, 2.438182458612187),
  (6, 2.618993567915248),
  (21, 2.649062380405664),
  (2, 2.7209119496855347),
  (15, 2.885368903709879),
  (0, 3.03330411919369),
  (3, 3.0399648660518226),
  (18, 3.345577573707105),
  (4, 3.490670700958144),
  (19, 4.117787031528852)],
 [(16, 1.0),
  (5, 1.1060070671378093),
  (12, 1.2084942084942085),
  (17, 1.3668122270742358),
  (20, 1.7936962750716332),
  (10, 1.8250728862973762),
  (22, 1.9202453987730062),
  (8, 1.926153846153846),
  (7, 2.039087947882736),
  (9, 2.045751633986928),
  (1, 2.079734219269103),
  (13, 2.114864864864865),
  (11, 2.129251700680272),
  (14, 2.1586206896551725),
 

## Establish a Log for Performance

In [20]:
performance_log = []

## Simple Baseline Model using `Conv1D`

In [21]:
def hamming_loss(y_true, y_pred, mode='multilabel'):
    if mode not in ['multiclass', 'multilabel']:
        raise TypeError('mode must be: [multiclass, multilabel])')

    if mode == 'multiclass':
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        print(nonzero)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), 
            tf.float32)
        return nonzero / y_true.get_shape()[-1]

In [22]:
import sys

## Model imports
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import ( Input, Embedding, GlobalMaxPooling1D, Conv1D, Dense, Activation, 
                                      Dropout, Lambda, BatchNormalization, concatenate )
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.text import Tokenizer

# Fit imports
from tensorflow.keras.losses import hinge, mae, binary_crossentropy, kld, Huber, squared_hinge

# Hyperparameter/method search space
import itertools


# For 4 GPUs
DIST_BATCH_SIZE = int(BATCH_SIZE/4)
EPOCHS = 3


print('Starting experiment loop...')

learning_rates = [0.01, 0.005, 0.001, 0.0005]
losses = [mae]# binary_crossentropy, mae, kld, Huber, hamming_loss]
activations = ['relu'] # 'selu'
optimizers = ['adam']
dropout_ratios = [0.2]
filter_lengths = [128]
class_weight_set = [None, train_class_weights]
sample_weight_set = [None, train_sample_weights]
test_sample_weight_set = [None]# , test_sample_weights]

args = list(itertools.product(
    learning_rates,
    losses,
    activations,
    optimizers,
    dropout_ratios,
    filter_lengths,
    class_weight_set,
    sample_weight_set,
    test_sample_weight_set
))
print()
print(f'{len(args):,} total iterations...')
sys.stdout.flush()

Starting experiment loop...

16 total iterations...


In [ ]:
# tqdm_notebook
for learning_rate, loss_function, activation, optimizer, dropout_ratio, filter_length, class_weights, \
    sample_weights, test_sample_weights in args:
    
    model_name = str(loss_function.__name__) + ' ' + str(learning_rate) + ' ' + str(optimizer) + ' ' + \
             str(activation) + ' ' + str(EPOCHS) + ' ' + \
             ('class_weights' if isinstance(class_weights, dict) else 'no_class_weights') + ' ' + \
             ('sample_weights' if isinstance(sample_weights, np.ndarray) else 'no_sample_weights') + ' ' + \
             ('test_sample_weights' if isinstance(test_sample_weights, list) else 'no_test_sample_weights')
    print(model_name)
    sys.stdout.flush()
    
    #
    # Build ze model...
    #
    def build_model(
        token_count=None,
        max_words=None,
        embedding_dim=None,
        label_count=None,
        dropout_ratio=None,
        filter_length=None,
        loss_function=None,
        learning_rate=None,
        optimizer=None,
        activation=None
    ):
        """Build the model using this experiment's parameters"""
        
        mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"])
        with mirrored_strategy.scope():
            
            print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))
        
            hashed_input = Input(shape=(X_train.shape[1],), dtype='int64')

            emb = Embedding(token_count, embedding_dim, weights=[embedding_matrix])(hashed_input)

            # Specify each convolution layer and their kernel siz i.e. n-grams 
            conv1_1 = Conv1D(filters=filter_length, kernel_size=3)(emb)
            btch1_1 = BatchNormalization()(conv1_1)
            drp1_1  = Dropout(dropout_ratio)(btch1_1)
            actv1_1 = Activation(activation)(drp1_1)
            glmp1_1 = GlobalMaxPooling1D()(actv1_1)

            conv1_2 = Conv1D(filters=filter_length, kernel_size=4)(emb)
            btch1_2 = BatchNormalization()(conv1_2)
            drp1_2  = Dropout(dropout_ratio)(btch1_2)
            actv1_2 = Activation(activation)(drp1_2)
            glmp1_2 = GlobalMaxPooling1D()(actv1_2)

            conv1_3 = Conv1D(filters=filter_length, kernel_size=5)(emb)
            btch1_3 = BatchNormalization()(conv1_3)
            drp1_3  = Dropout(dropout_ratio)(btch1_3)
            actv1_3 = Activation(activation)(drp1_3)
            glmp1_3 = GlobalMaxPooling1D()(actv1_3)

            conv1_4 = Conv1D(filters=filter_length, kernel_size=6)(emb)
            btch1_4 = BatchNormalization()(conv1_4)
            drp1_4  = Dropout(dropout_ratio)(btch1_4)
            actv1_4 = Activation(activation)(drp1_4)
            glmp1_4 = GlobalMaxPooling1D()(actv1_4)

            # Gather all convolution layers
            cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
            drp1 = Dropout(dropout_ratio)(cnct)

            dns1  = Dense(32, activation=activation)(drp1)
            btch1 = BatchNormalization()(dns1)
            drp2  = Dropout(dropout_ratio)(btch1)

            out = Dense(y_train.shape[1], activation='sigmoid')(drp2)

            text_model = Model(
                inputs=hashed_input, 
                outputs=out
            )

            if activation == 'adam':
                activation = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
            if activation == 'sgd':
                activation = SGD(lr=learning_rate)

            text_model.compile(
                optimizer=optimizer,
                loss=loss_function,
                metrics=[
                    'categorical_accuracy',
                    tf.keras.metrics.Precision(),
                    tf.keras.metrics.Recall(),
                    tf.keras.metrics.BinaryAccuracy(),
                    tf.keras.metrics.Hinge(),
                    tf.keras.metrics.AUC(),
                    tf.keras.metrics.Accuracy(),
                    tf.keras.metrics.MeanAbsoluteError(),
                    tf.keras.metrics.MeanAbsolutePercentageError(),
                    tf.keras.metrics.TruePositives(),
                    tf.keras.metrics.FalsePositives(),
                    tf.keras.metrics.TrueNegatives(),
                    tf.keras.metrics.FalseNegatives()
                ]
            )
            #text_model.summary()

            return text_model

    #
    # Train ze model...
    #
    def train_model(
        model=None,
        X_train=None,
        X_test=None,
        learning_rate=None,
        loss_function=None,
        optimizer=None,
        activation=None,
        epochs=None,
        class_weights=None,
        sample_weights=None,
        test_sample_weights=None,
    ):
        """Train the model using the current parameters and evaluate performance"""
        
        callbacks = [
            ReduceLROnPlateau(
                patience=1,
                verbose=1,
                min_delta=0.001,
                min_lr=0.0005,
            ), 
            EarlyStopping(
                patience=2,
                min_delta=0.001,
                verbose=1,
                restore_best_weights=True
            ), 
            #ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
        ]

        history = text_model.fit(
            X_train, 
            y_train,
            class_weight=class_weights,
            sample_weight=sample_weights,
            epochs=epochs,
            batch_size=DIST_BATCH_SIZE,
            validation_data=(X_test, y_test),
            callbacks=callbacks
        )
    
        # Evaluate to our log and return a description key and a list of metrics
        accr = text_model.evaluate(X_test, y_test, sample_weight=test_sample_weights)
        f1_score = 2.0 * (accr[1] * accr[2]) / \
                         (accr[1] + accr[2])
        return_val = [i for i in zip([j.item() for j in accr + [f1_score]], text_model.metrics_names + ['val_f1_score'])]

        return return_val

    #
    # main()
    #
    text_model = build_model(
        token_count=TOKEN_COUNT,
        max_words=100,
        embedding_dim=50,
        label_count=y_train.shape[1],
        filter_length=128,
        loss_function=loss_function,
        learning_rate=learning_rate,
        optimizer=optimizer,
        activation=activation,
        dropout_ratio=dropout_ratio
    )
    try:
        accuracies = train_model(
            model=text_model,
            X_train=X_train,
            X_test=X_test,
            learning_rate=learning_rate,
            loss_function=loss_function,
            optimizer=optimizer,
            activation=activation,
            epochs=EPOCHS,
            class_weights=class_weights,
            sample_weights=sample_weights if sample_weights is not None else sample_weights,
            test_sample_weights=test_sample_weights_sample if test_sample_weights is not None else test_sample_weights,
        )

        log_record = (model_name, accuracies)
        performance_log.append(log_record)

        with open('data/performance_log.jsonl', 'w') as f:
            for record in performance_log:
                f.write(json.dumps(record) + '\n')

        print(log_record)
        sys.stdout.flush()
    except KeyboardInterrupt as e:
        print('Aboring training run!')
        sys.stdout.flush()

print('Completed experiment loop!')

mean_absolute_error 0.01 adam relu 3 no_class_weights no_sample_weights no_test_sample_weights
Number of devices: 4


W0823 03:40:55.235537 140073442744128 deprecation.py:323] From /home/ubuntu/anaconda3/envs/deep/lib/python3.6/site-packages/tensorflow_core/python/keras/layers/normalization.py:625: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 100000 samples, validate on 10000 samples
Epoch 1/3


W0823 03:41:13.139191 140073442744128 cross_device_ops.py:779] Efficient allreduce is not supported for 1 IndexedSlices
W0823 03:41:22.046223 140073442744128 cross_device_ops.py:779] Efficient allreduce is not supported for 1 IndexedSlices


100000/100000 [==============================] - 237s 2ms/sample - loss: 0.0599 - categorical_accuracy: 0.0321 - precision: 0.0387 - recall: 0.0213 - binary_accuracy: 0.9512 - hinge: 1.0275 - auc: 0.5049 - accuracy: 0.0000e+00 - mean_absolute_error: 0.0599 - mean_absolute_percentage_error: 28568212.0000 - true_positives: 1656.0000 - false_positives: 41092.0000 - true_negatives: 2281333.0000 - false_negatives: 75919.0000 - val_loss: 0.0318 - val_categorical_accuracy: 0.0376 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_binary_accuracy: 0.9683 - val_hinge: 1.0001 - val_auc: 0.5000 - val_accuracy: 8.7500e-04 - val_mean_absolute_error: 0.0318 - val_mean_absolute_percentage_error: 88512.3281 - val_true_positives: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_true_negatives: 232391.0000 - val_false_negatives: 7609.0000
Epoch 2/3
 99968/100000 [============================>.] - ETA: 0s - loss: 0.0324 - categorical_accuracy: 0.0337 - precision: 0.0000e+00 - recall: 0.0000e+00

...

('mean_absolute_error 0.01 adam relu 3 class_weights no_sample_weights no_test_sample_weights', [(0.03178895898759365, 'loss'), (0.03819999843835831, 'categorical_accuracy'), (0.0, 'precision_2'), (0.0, 'recall_2'), (0.9682958722114563, 'binary_accuracy'), (1.0000851154327393, 'hinge'), (0.49998924136161804, 'auc_2'), (0.00027083331951871514, 'accuracy'), (0.031788960099220276, 'mean_absolute_error'), (87874.0703125, 'mean_absolute_percentage_error'), (0.0, 'true_positives_2'), (0.0, 'false_positives_2'), (232391.0, 'true_negatives_2'), (7609.0, 'false_negatives_2'), (0.0, 'val_f1_score')])
mean_absolute_error 0.01 adam relu 3 class_weights sample_weights no_test_sample_weights
Number of devices: 4
Train on 100000 samples, validate on 10000 samples
Epoch 1/3
100000/100000 [==============================] - 232s 2ms/sample - loss: 6.1053e-04 - categorical_accuracy: 0.0314 - precision_3: 0.0279 - recall_3: 0.3127 - binary_accuracy: 0.6258 - hinge: 1.3605 - auc_3: 0.4435 - accuracy: 0.000

('mean_absolute_error 0.01 adam relu 3 class_weights sample_weights no_test_sample_weights', [(0.10630255882740021, 'loss'), (0.03009999915957451, 'categorical_accuracy'), (0.09410183876752853, 'precision_3'), (0.19332370162010193, 'recall_3'), (0.9154207110404968, 'binary_accuracy'), (1.0745981931686401, 'hinge'), (0.5217179656028748, 'auc_3'), (1.249999968422344e-05, 'accuracy'), (0.10630256682634354, 'mean_absolute_error'), (80888568.0, 'mean_absolute_percentage_error'), (1471.0, 'true_positives_3'), (14161.0, 'false_positives_3'), (218230.0, 'true_negatives_3'), (6138.0, 'false_negatives_3'), (0.0456106845386779, 'val_f1_score')])
mean_absolute_error 0.005 adam relu 3 no_class_weights no_sample_weights no_test_sample_weights
Number of devices: 4
Train on 100000 samples, validate on 10000 samples
Epoch 1/3
100000/100000 [==============================] - 219s 2ms/sample - loss: 0.0594 - categorical_accuracy: 0.0439 - precision_4: 0.0351 - recall_4: 0.0190 - binary_accuracy: 0.9515 -

('mean_absolute_error 0.005 adam relu 3 no_class_weights no_sample_weights no_test_sample_weights', [(0.03178973667025566, 'loss'), (0.03400000184774399, 'categorical_accuracy'), (0.0, 'precision_4'), (0.0, 'recall_4'), (0.9682958722114563, 'binary_accuracy'), (1.0000869035720825, 'hinge'), (0.5, 'auc_4'), (0.0, 'accuracy'), (0.03178974241018295, 'mean_absolute_error'), (88673.5, 'mean_absolute_percentage_error'), (0.0, 'true_positives_4'), (0.0, 'false_positives_4'), (232391.0, 'true_negatives_4'), (7609.0, 'false_negatives_4'), (0.0, 'val_f1_score')])
mean_absolute_error 0.005 adam relu 3 no_class_weights sample_weights no_test_sample_weights
Number of devices: 4
Train on 100000 samples, validate on 10000 samples
Epoch 1/3
100000/100000 [==============================] - 219s 2ms/sample - loss: 2.8784e-04 - categorical_accuracy: 0.0243 - precision_5: 0.0284 - recall_5: 0.3218 - binary_accuracy: 0.6222 - hinge: 1.3738 - auc_5: 0.4508 - accuracy: 0.0000e+00 - mean_absolute_error: 0.406

('mean_absolute_error 0.005 adam relu 3 no_class_weights sample_weights no_test_sample_weights', [(0.13986727323532105, 'loss'), (0.037300001829862595, 'categorical_accuracy'), (0.054307810962200165, 'precision_5'), (0.2113286852836609, 'recall_5'), (0.8583248257637024, 'binary_accuracy'), (1.1081632375717163, 'hinge'), (0.5354441404342651, 'auc_5'), (0.0, 'accuracy'), (0.13986724615097046, 'mean_absolute_error'), (114885536.0, 'mean_absolute_percentage_error'), (1608.0, 'true_positives_5'), (28001.0, 'false_positives_5'), (204390.0, 'true_negatives_5'), (6001.0, 'false_negatives_5'), (0.044225084486936246, 'val_f1_score')])
mean_absolute_error 0.005 adam relu 3 class_weights no_sample_weights no_test_sample_weights
Number of devices: 4
Train on 100000 samples, validate on 10000 samples
Epoch 1/3
100000/100000 [==============================] - 232s 2ms/sample - loss: 0.1487 - categorical_accuracy: 0.0488 - precision_6: 0.0452 - recall_6: 0.0293 - binary_accuracy: 0.9486 - hinge: 1.029

('mean_absolute_error 0.005 adam relu 3 class_weights no_sample_weights no_test_sample_weights', [(0.031785713478922846, 'loss'), (0.034299999475479126, 'categorical_accuracy'), (0.0, 'precision_6'), (0.0, 'recall_6'), (0.9682958722114563, 'binary_accuracy'), (1.00008225440979, 'hinge'), (0.5, 'auc_6'), (0.0, 'accuracy'), (0.03178570419549942, 'mean_absolute_error'), (84751.0546875, 'mean_absolute_percentage_error'), (0.0, 'true_positives_6'), (0.0, 'false_positives_6'), (232391.0, 'true_negatives_6'), (7609.0, 'false_negatives_6'), (0.0, 'val_f1_score')])
mean_absolute_error 0.005 adam relu 3 class_weights sample_weights no_test_sample_weights
Number of devices: 4
Train on 100000 samples, validate on 10000 samples
Epoch 1/3
100000/100000 [==============================] - 230s 2ms/sample - loss: 5.8165e-04 - categorical_accuracy: 0.0281 - precision_7: 0.0251 - recall_7: 0.3429 - binary_accuracy: 0.5484 - hinge: 1.4154 - auc_7: 0.4291 - accuracy: 4.1667e-07 - mean_absolute_error: 0.447

In [ ]:
performance_log

In [ ]:
# from keras.callbacks import EarlyStopping

# EPOCHS = 4

# history = text_model.fit(
#     X_train,
#     y_train,
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     callbacks=[
#         EarlyStopping(monitor='loss', patience=1, min_delta=0.0001),
#         EarlyStopping(monitor='val_loss', patience=1, min_delta=0.0001),
#     ],
#     class_weight=class_weights,
#     # sample_weight=train_sample_weights,
#     validation_data=(X_test, y_test)
# )

In [ ]:
accr = text_model.evaluate(X_test, y_test) #, sample_weight=test_sample_weights)
[i for i in zip(accr, text_model.metrics_names)]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

print(history.history)
# summarize history for accuracy
plt.plot(history.history['val_loss'])
plt.plot(history.history['f1_m'])
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['precision_m'])
plt.title('model accuracy')
plt.ylabel('metric')
plt.xlabel('epoch')
plt.legend(['val_loss', 'f1', 'categorical accuracy', 'MAE', 'precision'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import statistics

from sklearn.metrics import hamming_loss, jaccard_score
import keras.backend as K
import tensorflow as tf

y_pred = text_model.predict(X_test)

sess = tf.Session()
best_cutoff = 0
max_score = 0
with sess.as_default():
    for cutoff in [0.0001, 0.001, 0.01, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
        y_pred_bin = K.greater(y_pred, cutoff).eval()
        print('Cutoff: {:,}'.format(cutoff))
        print('Hamming loss: {:,}'.format(
            hamming_loss(y_test, y_pred_bin)
        ))
        scores = []
        for j_type in ['micro', 'macro', 'weighted']:
            j_score = jaccard_score(y_test, y_pred_bin, average=j_type)
            print('Jaccard {} score: {:,}'.format(
                j_type,
                j_score
            ))
            scores.append(j_score)
        print('')
        mean_score = statistics.mean(scores)
        if mean_score > max_score:
            best_cutoff = cutoff
            max_score = mean_score

print('Best cutoff was: {:,} with mean jaccard score of {:,}'.format(best_cutoff, max_score))

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

y_pred = text_model.predict(X_test, batch_size=32, verbose=1)
y_pred_bool = np.where(y_pred > best_cutoff, 1, 0)

print(classification_report(y_test, y_pred_bool))

print(multilabel_confusion_matrix(y_test, y_pred_bool))

## View the Results

Now lets map from the one-hot-encoded tags back to the text tags and view them alongside the text of the original posts to sanity check the model and see if it really works.

In [ ]:
predicted_tags = []
for test, pred in zip(y_test, y_pred_bool):
    tags = []
    for i, val in enumerate(test):
        if pred[i] == 1.0:
            tags.append(sorted_all_tags[i])
    predicted_tags.append(tags)

for text, tags in zip(X_test, predicted_tags):
    print(' '.join(text), tags)